In [3]:
import io
import os
import math
import stackstac
import geojson
import dask_gateway
import planetary_computer
import rasterio.features
import azure.storage.blob
import numpy as np
import xarray as xr
import rioxarray as rioxr
import pysptools.abundance_maps as amp
import matplotlib.pyplot as plt
from dask.distributed import PipInstall, Lock
from scipy.stats import mode
from dask_gateway import GatewayCluster
from pystac_client import Client
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [4]:
def install():
    os.system('pip install pysptools') 
    os.system('pip install cvxopt')  

def read_file(file):
    with open(file) as f:
        content = f.read()
    return content

def load_blob_grid(blob_client):
    return geojson.loads(blob_client.download_blob().readall())

def get_tile(grid, h, v):
    return [x for x in grid['features'] if x['properties']['horizontal'] == h
            and x['properties']['vertical'] == v][0]['geometry']

def get_bbox(geometry):
    return rasterio.features.bounds(tile)

def get_container(container, connection_string):
    container_client = azure.storage.blob.ContainerClient.from_connection_string(
        connection_string, container_name=container
    )
    return container_client

def get_blob(container_client, blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    return blob_client

def get_cluster(n=20, ncore=8, memory=16):
    gateway = dask_gateway.Gateway()
    cluster_options = gateway.cluster_options()
    cluster_options["worker_cores"] = ncore
    cluster_options["worker_memory"] = memory
    
    cluster = gateway.new_cluster(cluster_options)
    cluster.scale(n)
    client = cluster.get_client()
    return (cluster, client)

def register_package():
    plugin = PipInstall(packages=['pysptools', 'cvxopt'], pip_options=['--upgrade'])
    client.register_worker_plugin(plugin)

endmembers = np.array([[500, 900, 400, 6100, 3000, 1000],
                       [1400, 1700, 2200, 3000, 5500, 3000],
                       [2000, 3000, 3400, 5800, 6000, 5800],
                       [0, 0, 0, 0, 0, 0],
                       [9000, 9600, 8000, 7800, 7200, 6500]], dtype=np.int16)

In [30]:
def search_sentinel2_images(start, end, geometry, limit=1000):
    search = catalog.search(
        bbox = geometry,
        datetime = start + '/' + end,
        collections = ['sentinel-2-l2a'],
        limit = 1000,
        query={'eo:cloud_cover': {'lt': 90}}
    )
    return list(search.get_items())

def get_sentinel2_stack(start, end, geometry, chunksize=1024):
    items = search_sentinel2_images(start, end, geometry)
    signed_items = [planetary_computer.sign(item).to_dict() for item in items]
    
    #bbox = get_bbox(geometry)
    epsg = get_epsg(items)
    
    data = (
        stackstac.stack(
            signed_items,
            assets=['blue'],
            #assets=['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'],
            #chunksize=(1, 1, chunksize, chunksize),
            resolution=30,
            #epsg=epsg,
            #bounds_latlon=bbox
        )
        #.assign_coords(band=['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'QA'])
    )
    return data#.chunk((-1, -1, chunksize, chunksize))\

In [31]:
connection_string = read_file('/home/jovyan/fnrtm/files/connect.txt')
container_client = get_container('misc', connection_string)
blob_client = get_blob(container_client, 'amazon_grid.geojson')
training_container = get_container('training', connection_string)
grid_blob = load_blob_grid(blob_client)
tile = get_tile(grid_blob, 41, 28)

In [32]:
catalog = Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')
#catalog = Client.open('https://planetarycomputer-staging.microsoft.com/api/stac/v1')

def get_epsg(items):
    epsgs = [x.properties['proj:epsg'] for x in items]
    return 'EPSG: ' + str(mode(epsgs).mode[0])

In [33]:
tile
bbox = get_bbox(tile)
bbox

(-67.008753, -9.96445, -65.615556, -8.57408)

In [34]:
s2 = get_sentinel2_stack('2020-06-01', '2020-12-31', bbox)
s2

/tmp/ipykernel_1175/732763501.py:6: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return 'EPSG: ' + str(mode(epsgs).mode[0])


TypeError: cannot unpack non-iterable NoneType object